# tratando_anomalias
En el TP1 hemos visto que el dataset, aparte de datos sin sentido o datos nulos, presenta anomalias, y no solo eso, sino que se trata de un dataset desbalanceado.

Decimos que presenta anomalias y que es un dataset desbalanceado porque:
- Se cargaron muchas mas propiedades en **Diciembre del 2016** que en el resto de los anio-meses. Consideramos que esto es una **anomalia** del dataset, ya que no hay razon alguna para que en el ultimo mes los datos se cuadripliquen. Sospechamos que pueden ser datos que no correspondan a ese anio-mes.
- Algo similar sucede con las provincias, donde **Distrito Federal** tiene la mayoria de publicaciones. Esto no es una anomalia ya que es la provincia mas poblada de Mexico y tiene sentido que tenga mas publicaciones. Aun asi, genera **desbalanceo** en el dataset, y puede provocar que algunos algoritmos funcionen peor.

En las siguientes lineas intentaremos proponer algunas soluciones a estos problemas:

In [1]:
import pandas as pd
import numpy as np
import math

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

seed = 7

In [2]:
import nbimporter

import pre_processing
import feature_generation
import feature_selection

Importing Jupyter notebook from pre_processing.ipynb
Importing Jupyter notebook from feature_generation.ipynb
Importing Jupyter notebook from feature_selection.ipynb


<hr>

### Modelo a utilizar:

In [3]:
import lightgbm as lgb

params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': 'mae',
    'max_depth': 14, 
    'learning_rate': 0.1,
    'verbose': 0, 
    'early_stopping_round': 50,
    'n_jobs':2}
n_estimators=10000

<hr>

# TRATAMIENTO DE ANIO-MES DIC2016

## Primera alternativa: llenar con nulos los campos de fecha para dichos datos.
Analizaremos esta opcion probando realizar el tratamiento solo para el dataset de train, y luego para ambos y ver que resultados obtenemos.

In [4]:
train,_ = pre_processing.load_featured_datasets()

In [5]:
features = feature_generation.get_features()
features_con_fecha = features['fecha']['all']

In [6]:
train = feature_selection.get_selected_dataframe(train)

In [7]:
train.shape

(240000, 152)

In [8]:
train['precio'] = train['precio'].map(lambda x: math.log(x))
train_original = train.copy()

In [9]:
for feature in features_con_fecha:
    if feature in train.columns:
        train[feature] = train.apply(lambda x: (np.nan) if (x['aniomes'] == 201612) else (x[feature]), axis=1)

In [10]:
X = train.drop('precio', axis=1)
Y = train['precio']

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=seed)

In [11]:
d_train = lgb.Dataset(X_train.values, label=Y_train.values)
d_valid = lgb.Dataset(X_val.values, label=Y_val.values)
watchlist = [d_valid]
reg = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=1)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l1: 0.637798
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 0.590846
[3]	valid_0's l1: 0.549523
[4]	valid_0's l1: 0.51271
[5]	valid_0's l1: 0.480274
[6]	valid_0's l1: 0.451318
[7]	valid_0's l1: 0.426066
[8]	valid_0's l1: 0.403509
[9]	valid_0's l1: 0.383537
[10]	valid_0's l1: 0.365999
[11]	valid_0's l1: 0.350865
[12]	valid_0's l1: 0.337515
[13]	valid_0's l1: 0.325643
[14]	valid_0's l1: 0.314825
[15]	valid_0's l1: 0.305496
[16]	valid_0's l1: 0.297011
[17]	valid_0's l1: 0.289464
[18]	valid_0's l1: 0.282812
[19]	valid_0's l1: 0.27694
[20]	valid_0's l1: 0.272012
[21]	valid_0's l1: 0.267315
[22]	valid_0's l1: 0.263355
[23]	valid_0's l1: 0.259511
[24]	valid_0's l1: 0.256279
[25]	valid_0's l1: 0.253376
[26]	valid_0's l1: 0.250684
[27]	valid_0's l1: 0.248168
[28]	valid_0's l1: 0.246041
[29]	valid_0's l1: 0.244057
[30]	valid_0's l1: 0.242332
[31]	valid_0's l1: 0.240692
[32]	valid_0's l1: 0.239176
[33]	valid_0's l1: 0.237821
[34]	valid_0's l1: 0.23654

[289]	valid_0's l1: 0.203269
[290]	valid_0's l1: 0.203254
[291]	valid_0's l1: 0.203221
[292]	valid_0's l1: 0.203206
[293]	valid_0's l1: 0.203119
[294]	valid_0's l1: 0.203067
[295]	valid_0's l1: 0.203009
[296]	valid_0's l1: 0.202989
[297]	valid_0's l1: 0.202959
[298]	valid_0's l1: 0.202933
[299]	valid_0's l1: 0.202899
[300]	valid_0's l1: 0.202893
[301]	valid_0's l1: 0.202866
[302]	valid_0's l1: 0.202847
[303]	valid_0's l1: 0.202775
[304]	valid_0's l1: 0.202738
[305]	valid_0's l1: 0.202726
[306]	valid_0's l1: 0.202731
[307]	valid_0's l1: 0.202695
[308]	valid_0's l1: 0.20267
[309]	valid_0's l1: 0.202611
[310]	valid_0's l1: 0.202612
[311]	valid_0's l1: 0.202593
[312]	valid_0's l1: 0.202586
[313]	valid_0's l1: 0.202561
[314]	valid_0's l1: 0.202534
[315]	valid_0's l1: 0.202484
[316]	valid_0's l1: 0.202436
[317]	valid_0's l1: 0.202399
[318]	valid_0's l1: 0.202351
[319]	valid_0's l1: 0.202329
[320]	valid_0's l1: 0.202296
[321]	valid_0's l1: 0.202267
[322]	valid_0's l1: 0.202237
[323]	valid_0's

[573]	valid_0's l1: 0.197855
[574]	valid_0's l1: 0.197845
[575]	valid_0's l1: 0.197833
[576]	valid_0's l1: 0.19782
[577]	valid_0's l1: 0.197772
[578]	valid_0's l1: 0.197756
[579]	valid_0's l1: 0.197751
[580]	valid_0's l1: 0.19774
[581]	valid_0's l1: 0.197732
[582]	valid_0's l1: 0.197718
[583]	valid_0's l1: 0.197709
[584]	valid_0's l1: 0.197697
[585]	valid_0's l1: 0.197701
[586]	valid_0's l1: 0.197686
[587]	valid_0's l1: 0.197679
[588]	valid_0's l1: 0.197647
[589]	valid_0's l1: 0.197618
[590]	valid_0's l1: 0.197601
[591]	valid_0's l1: 0.197588
[592]	valid_0's l1: 0.197576
[593]	valid_0's l1: 0.197574
[594]	valid_0's l1: 0.197573
[595]	valid_0's l1: 0.197564
[596]	valid_0's l1: 0.197555
[597]	valid_0's l1: 0.197544
[598]	valid_0's l1: 0.197541
[599]	valid_0's l1: 0.19753
[600]	valid_0's l1: 0.197502
[601]	valid_0's l1: 0.197479
[602]	valid_0's l1: 0.197463
[603]	valid_0's l1: 0.197452
[604]	valid_0's l1: 0.19745
[605]	valid_0's l1: 0.19745
[606]	valid_0's l1: 0.197428
[607]	valid_0's l1:

[863]	valid_0's l1: 0.195032
[864]	valid_0's l1: 0.195008
[865]	valid_0's l1: 0.195009
[866]	valid_0's l1: 0.194998
[867]	valid_0's l1: 0.194999
[868]	valid_0's l1: 0.194986
[869]	valid_0's l1: 0.194984
[870]	valid_0's l1: 0.194976
[871]	valid_0's l1: 0.194968
[872]	valid_0's l1: 0.194963
[873]	valid_0's l1: 0.194955
[874]	valid_0's l1: 0.194947
[875]	valid_0's l1: 0.194943
[876]	valid_0's l1: 0.194938
[877]	valid_0's l1: 0.194924
[878]	valid_0's l1: 0.194908
[879]	valid_0's l1: 0.194902
[880]	valid_0's l1: 0.19489
[881]	valid_0's l1: 0.194888
[882]	valid_0's l1: 0.194888
[883]	valid_0's l1: 0.194889
[884]	valid_0's l1: 0.19488
[885]	valid_0's l1: 0.194876
[886]	valid_0's l1: 0.194878
[887]	valid_0's l1: 0.194878
[888]	valid_0's l1: 0.194876
[889]	valid_0's l1: 0.194862
[890]	valid_0's l1: 0.194864
[891]	valid_0's l1: 0.194858
[892]	valid_0's l1: 0.194854
[893]	valid_0's l1: 0.194834
[894]	valid_0's l1: 0.194827
[895]	valid_0's l1: 0.1948
[896]	valid_0's l1: 0.194786
[897]	valid_0's l1

...

[2544]	valid_0's l1: 0.189311
[2545]	valid_0's l1: 0.189313
[2546]	valid_0's l1: 0.189312
[2547]	valid_0's l1: 0.189313
[2548]	valid_0's l1: 0.189311
[2549]	valid_0's l1: 0.189306
[2550]	valid_0's l1: 0.189298
[2551]	valid_0's l1: 0.189298
[2552]	valid_0's l1: 0.189293
[2553]	valid_0's l1: 0.189293
[2554]	valid_0's l1: 0.189288
[2555]	valid_0's l1: 0.18929
[2556]	valid_0's l1: 0.189285
[2557]	valid_0's l1: 0.189283
[2558]	valid_0's l1: 0.189281
[2559]	valid_0's l1: 0.189272
[2560]	valid_0's l1: 0.189271
[2561]	valid_0's l1: 0.189273
[2562]	valid_0's l1: 0.189272
[2563]	valid_0's l1: 0.189271
[2564]	valid_0's l1: 0.189267
[2565]	valid_0's l1: 0.189269
[2566]	valid_0's l1: 0.189267
[2567]	valid_0's l1: 0.189266
[2568]	valid_0's l1: 0.189266
[2569]	valid_0's l1: 0.189263
[2570]	valid_0's l1: 0.189269
[2571]	valid_0's l1: 0.189258
[2572]	valid_0's l1: 0.189258
[2573]	valid_0's l1: 0.189262
[2574]	valid_0's l1: 0.189255
[2575]	valid_0's l1: 0.189255
[2576]	valid_0's l1: 0.189257
[2577]	vali

[2827]	valid_0's l1: 0.188895
[2828]	valid_0's l1: 0.188892
[2829]	valid_0's l1: 0.188891
[2830]	valid_0's l1: 0.188894
[2831]	valid_0's l1: 0.188891
[2832]	valid_0's l1: 0.188892
[2833]	valid_0's l1: 0.188895
[2834]	valid_0's l1: 0.188894
[2835]	valid_0's l1: 0.188894
[2836]	valid_0's l1: 0.188893
[2837]	valid_0's l1: 0.188894
[2838]	valid_0's l1: 0.188896
[2839]	valid_0's l1: 0.188892
[2840]	valid_0's l1: 0.188892
[2841]	valid_0's l1: 0.188893
[2842]	valid_0's l1: 0.188889
[2843]	valid_0's l1: 0.188887
[2844]	valid_0's l1: 0.188891
[2845]	valid_0's l1: 0.188886
[2846]	valid_0's l1: 0.188882
[2847]	valid_0's l1: 0.188885
[2848]	valid_0's l1: 0.188883
[2849]	valid_0's l1: 0.188884
[2850]	valid_0's l1: 0.188872
[2851]	valid_0's l1: 0.188878
[2852]	valid_0's l1: 0.188875
[2853]	valid_0's l1: 0.188869
[2854]	valid_0's l1: 0.18887
[2855]	valid_0's l1: 0.188864
[2856]	valid_0's l1: 0.188859
[2857]	valid_0's l1: 0.188862
[2858]	valid_0's l1: 0.188862
[2859]	valid_0's l1: 0.188863
[2860]	vali

[3108]	valid_0's l1: 0.188563
[3109]	valid_0's l1: 0.18856
[3110]	valid_0's l1: 0.188562
[3111]	valid_0's l1: 0.188559
[3112]	valid_0's l1: 0.188559
[3113]	valid_0's l1: 0.188563
[3114]	valid_0's l1: 0.188565
[3115]	valid_0's l1: 0.18856
[3116]	valid_0's l1: 0.188555
[3117]	valid_0's l1: 0.188558
[3118]	valid_0's l1: 0.188561
[3119]	valid_0's l1: 0.188561
[3120]	valid_0's l1: 0.188561
[3121]	valid_0's l1: 0.188558
[3122]	valid_0's l1: 0.188561
[3123]	valid_0's l1: 0.188552
[3124]	valid_0's l1: 0.188552
[3125]	valid_0's l1: 0.188555
[3126]	valid_0's l1: 0.188559
[3127]	valid_0's l1: 0.188564
[3128]	valid_0's l1: 0.188564
[3129]	valid_0's l1: 0.188564
[3130]	valid_0's l1: 0.188562
[3131]	valid_0's l1: 0.188561
[3132]	valid_0's l1: 0.188562
[3133]	valid_0's l1: 0.188563
[3134]	valid_0's l1: 0.188562
[3135]	valid_0's l1: 0.188559
[3136]	valid_0's l1: 0.188558
[3137]	valid_0's l1: 0.188561
[3138]	valid_0's l1: 0.188562
[3139]	valid_0's l1: 0.188562
[3140]	valid_0's l1: 0.188559
[3141]	valid

[3389]	valid_0's l1: 0.188252
[3390]	valid_0's l1: 0.188248
[3391]	valid_0's l1: 0.188242
[3392]	valid_0's l1: 0.188238
[3393]	valid_0's l1: 0.188233
[3394]	valid_0's l1: 0.188229
[3395]	valid_0's l1: 0.188223
[3396]	valid_0's l1: 0.188223
[3397]	valid_0's l1: 0.188223
[3398]	valid_0's l1: 0.188223
[3399]	valid_0's l1: 0.188222
[3400]	valid_0's l1: 0.188222
[3401]	valid_0's l1: 0.188219
[3402]	valid_0's l1: 0.188216
[3403]	valid_0's l1: 0.188211
[3404]	valid_0's l1: 0.188204
[3405]	valid_0's l1: 0.18821
[3406]	valid_0's l1: 0.188204
[3407]	valid_0's l1: 0.188201
[3408]	valid_0's l1: 0.188193
[3409]	valid_0's l1: 0.188195
[3410]	valid_0's l1: 0.188196
[3411]	valid_0's l1: 0.188197
[3412]	valid_0's l1: 0.188199
[3413]	valid_0's l1: 0.188206
[3414]	valid_0's l1: 0.188207
[3415]	valid_0's l1: 0.188206
[3416]	valid_0's l1: 0.188207
[3417]	valid_0's l1: 0.18821
[3418]	valid_0's l1: 0.18821
[3419]	valid_0's l1: 0.188212
[3420]	valid_0's l1: 0.188211
[3421]	valid_0's l1: 0.188208
[3422]	valid_

[3668]	valid_0's l1: 0.187968
[3669]	valid_0's l1: 0.187969
[3670]	valid_0's l1: 0.187968
[3671]	valid_0's l1: 0.187968
[3672]	valid_0's l1: 0.187966
[3673]	valid_0's l1: 0.187965
[3674]	valid_0's l1: 0.187963
[3675]	valid_0's l1: 0.18796
[3676]	valid_0's l1: 0.187957
[3677]	valid_0's l1: 0.187954
[3678]	valid_0's l1: 0.187952
[3679]	valid_0's l1: 0.187953
[3680]	valid_0's l1: 0.18795
[3681]	valid_0's l1: 0.187949
[3682]	valid_0's l1: 0.187952
[3683]	valid_0's l1: 0.18795
[3684]	valid_0's l1: 0.187948
[3685]	valid_0's l1: 0.18795
[3686]	valid_0's l1: 0.187951
[3687]	valid_0's l1: 0.187947
[3688]	valid_0's l1: 0.187942
[3689]	valid_0's l1: 0.187943
[3690]	valid_0's l1: 0.187938
[3691]	valid_0's l1: 0.187937
[3692]	valid_0's l1: 0.187936
[3693]	valid_0's l1: 0.187931
[3694]	valid_0's l1: 0.187933
[3695]	valid_0's l1: 0.187937
[3696]	valid_0's l1: 0.187936
[3697]	valid_0's l1: 0.187932
[3698]	valid_0's l1: 0.187925
[3699]	valid_0's l1: 0.187924
[3700]	valid_0's l1: 0.187918
[3701]	valid_0

In [12]:
# Con la mejora:

Y_pred = reg.predict(X_val.values)

f = np.vectorize(math.exp)
Y_pred = f(Y_pred)
Y_val = f(Y_val.values)
mean_absolute_error(Y_val,Y_pred)

471747.25660757907

In [13]:
# Ahora, para comparar, probaremos el mismo metodo pero sin mejora aplicada:

In [14]:
X = train_original.drop('precio', axis=1)
Y = train_original['precio']

X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=seed)

In [15]:
d_train = lgb.Dataset(X_train.values, label=Y_train.values)
d_valid = lgb.Dataset(X_val.values, label=Y_val.values)
watchlist = [d_valid]
reg = lgb.train(params, d_train, n_estimators, watchlist, verbose_eval=1)

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's l1: 0.637798
Training until validation scores don't improve for 50 rounds
[2]	valid_0's l1: 0.590846
[3]	valid_0's l1: 0.549523
[4]	valid_0's l1: 0.51271
[5]	valid_0's l1: 0.480274
[6]	valid_0's l1: 0.451318
[7]	valid_0's l1: 0.426066
[8]	valid_0's l1: 0.403509
[9]	valid_0's l1: 0.383537
[10]	valid_0's l1: 0.365999
[11]	valid_0's l1: 0.350865
[12]	valid_0's l1: 0.337515
[13]	valid_0's l1: 0.325643
[14]	valid_0's l1: 0.314825
[15]	valid_0's l1: 0.305496
[16]	valid_0's l1: 0.297011
[17]	valid_0's l1: 0.289464
[18]	valid_0's l1: 0.282812
[19]	valid_0's l1: 0.27694
[20]	valid_0's l1: 0.272012
[21]	valid_0's l1: 0.267315
[22]	valid_0's l1: 0.263355
[23]	valid_0's l1: 0.259511
[24]	valid_0's l1: 0.256279
[25]	valid_0's l1: 0.253376
[26]	valid_0's l1: 0.250684
[27]	valid_0's l1: 0.248168
[28]	valid_0's l1: 0.246041
[29]	valid_0's l1: 0.244057
[30]	valid_0's l1: 0.242332
[31]	valid_0's l1: 0.240692
[32]	valid_0's l1: 0.239176
[33]	valid_0's l1: 0.237821
[34]	valid_0's l1: 0.23654

[294]	valid_0's l1: 0.203124
[295]	valid_0's l1: 0.203085
[296]	valid_0's l1: 0.203025
[297]	valid_0's l1: 0.203006
[298]	valid_0's l1: 0.202983
[299]	valid_0's l1: 0.202961
[300]	valid_0's l1: 0.202948
[301]	valid_0's l1: 0.202927
[302]	valid_0's l1: 0.202909
[303]	valid_0's l1: 0.202887
[304]	valid_0's l1: 0.202843
[305]	valid_0's l1: 0.202779
[306]	valid_0's l1: 0.202756
[307]	valid_0's l1: 0.202741
[308]	valid_0's l1: 0.202717
[309]	valid_0's l1: 0.202692
[310]	valid_0's l1: 0.202677
[311]	valid_0's l1: 0.202672
[312]	valid_0's l1: 0.202663
[313]	valid_0's l1: 0.202648
[314]	valid_0's l1: 0.202625
[315]	valid_0's l1: 0.202564
[316]	valid_0's l1: 0.202552
[317]	valid_0's l1: 0.202538
[318]	valid_0's l1: 0.202515
[319]	valid_0's l1: 0.202509
[320]	valid_0's l1: 0.202499
[321]	valid_0's l1: 0.202472
[322]	valid_0's l1: 0.202465
[323]	valid_0's l1: 0.202449
[324]	valid_0's l1: 0.202434
[325]	valid_0's l1: 0.20237
[326]	valid_0's l1: 0.20235
[327]	valid_0's l1: 0.202338
[328]	valid_0's 

[583]	valid_0's l1: 0.197962
[584]	valid_0's l1: 0.197952
[585]	valid_0's l1: 0.197946
[586]	valid_0's l1: 0.197941
[587]	valid_0's l1: 0.197927
[588]	valid_0's l1: 0.197915
[589]	valid_0's l1: 0.197869
[590]	valid_0's l1: 0.197832
[591]	valid_0's l1: 0.197816
[592]	valid_0's l1: 0.197806
[593]	valid_0's l1: 0.197788
[594]	valid_0's l1: 0.197778
[595]	valid_0's l1: 0.197746
[596]	valid_0's l1: 0.197749
[597]	valid_0's l1: 0.197752
[598]	valid_0's l1: 0.197738
[599]	valid_0's l1: 0.197698
[600]	valid_0's l1: 0.197666
[601]	valid_0's l1: 0.19766
[602]	valid_0's l1: 0.197659
[603]	valid_0's l1: 0.197656
[604]	valid_0's l1: 0.197644
[605]	valid_0's l1: 0.197627
[606]	valid_0's l1: 0.197629
[607]	valid_0's l1: 0.197624
[608]	valid_0's l1: 0.197618
[609]	valid_0's l1: 0.197611
[610]	valid_0's l1: 0.197596
[611]	valid_0's l1: 0.197591
[612]	valid_0's l1: 0.197585
[613]	valid_0's l1: 0.197566
[614]	valid_0's l1: 0.197557
[615]	valid_0's l1: 0.197533
[616]	valid_0's l1: 0.19753
[617]	valid_0's 

[871]	valid_0's l1: 0.194979
[872]	valid_0's l1: 0.194968
[873]	valid_0's l1: 0.194961
[874]	valid_0's l1: 0.194937
[875]	valid_0's l1: 0.194918
[876]	valid_0's l1: 0.194915
[877]	valid_0's l1: 0.194917
[878]	valid_0's l1: 0.19491
[879]	valid_0's l1: 0.194901
[880]	valid_0's l1: 0.194903
[881]	valid_0's l1: 0.194903
[882]	valid_0's l1: 0.194895
[883]	valid_0's l1: 0.194895
[884]	valid_0's l1: 0.194887
[885]	valid_0's l1: 0.194874
[886]	valid_0's l1: 0.194857
[887]	valid_0's l1: 0.194847
[888]	valid_0's l1: 0.194848
[889]	valid_0's l1: 0.19484
[890]	valid_0's l1: 0.194836
[891]	valid_0's l1: 0.194836
[892]	valid_0's l1: 0.194835
[893]	valid_0's l1: 0.19483
[894]	valid_0's l1: 0.194827
[895]	valid_0's l1: 0.194816
[896]	valid_0's l1: 0.194799
[897]	valid_0's l1: 0.194797
[898]	valid_0's l1: 0.194792
[899]	valid_0's l1: 0.194785
[900]	valid_0's l1: 0.19478
[901]	valid_0's l1: 0.194765
[902]	valid_0's l1: 0.194765
[903]	valid_0's l1: 0.194751
[904]	valid_0's l1: 0.194741
[905]	valid_0's l1

...

[3383]	valid_0's l1: 0.187973
[3384]	valid_0's l1: 0.187977
[3385]	valid_0's l1: 0.187974
[3386]	valid_0's l1: 0.187973
[3387]	valid_0's l1: 0.187971
[3388]	valid_0's l1: 0.187966
[3389]	valid_0's l1: 0.187967
[3390]	valid_0's l1: 0.187966
[3391]	valid_0's l1: 0.187967
[3392]	valid_0's l1: 0.187972
[3393]	valid_0's l1: 0.187971
[3394]	valid_0's l1: 0.18797
[3395]	valid_0's l1: 0.187969
[3396]	valid_0's l1: 0.187965
[3397]	valid_0's l1: 0.187964
[3398]	valid_0's l1: 0.187965
[3399]	valid_0's l1: 0.187964
[3400]	valid_0's l1: 0.187963
[3401]	valid_0's l1: 0.187965
[3402]	valid_0's l1: 0.187964
[3403]	valid_0's l1: 0.187953
[3404]	valid_0's l1: 0.187952
[3405]	valid_0's l1: 0.187952
[3406]	valid_0's l1: 0.187946
[3407]	valid_0's l1: 0.187949
[3408]	valid_0's l1: 0.18795
[3409]	valid_0's l1: 0.18795
[3410]	valid_0's l1: 0.187945
[3411]	valid_0's l1: 0.187946
[3412]	valid_0's l1: 0.187947
[3413]	valid_0's l1: 0.187945
[3414]	valid_0's l1: 0.187947
[3415]	valid_0's l1: 0.187946
[3416]	valid_

[3663]	valid_0's l1: 0.187761
[3664]	valid_0's l1: 0.187761
[3665]	valid_0's l1: 0.187756
[3666]	valid_0's l1: 0.187753
[3667]	valid_0's l1: 0.18775
[3668]	valid_0's l1: 0.187746
[3669]	valid_0's l1: 0.187745
[3670]	valid_0's l1: 0.187742
[3671]	valid_0's l1: 0.187741
[3672]	valid_0's l1: 0.187738
[3673]	valid_0's l1: 0.187737
[3674]	valid_0's l1: 0.187739
[3675]	valid_0's l1: 0.187743
[3676]	valid_0's l1: 0.187744
[3677]	valid_0's l1: 0.187741
[3678]	valid_0's l1: 0.187743
[3679]	valid_0's l1: 0.187744
[3680]	valid_0's l1: 0.187741
[3681]	valid_0's l1: 0.187737
[3682]	valid_0's l1: 0.187736
[3683]	valid_0's l1: 0.187731
[3684]	valid_0's l1: 0.187731
[3685]	valid_0's l1: 0.187728
[3686]	valid_0's l1: 0.187728
[3687]	valid_0's l1: 0.187721
[3688]	valid_0's l1: 0.187719
[3689]	valid_0's l1: 0.187713
[3690]	valid_0's l1: 0.187712
[3691]	valid_0's l1: 0.18771
[3692]	valid_0's l1: 0.18771
[3693]	valid_0's l1: 0.187709
[3694]	valid_0's l1: 0.187711
[3695]	valid_0's l1: 0.18771
[3696]	valid_0

[3941]	valid_0's l1: 0.18757
[3942]	valid_0's l1: 0.187573
[3943]	valid_0's l1: 0.187577
[3944]	valid_0's l1: 0.187575
[3945]	valid_0's l1: 0.187573
[3946]	valid_0's l1: 0.187571
[3947]	valid_0's l1: 0.187575
[3948]	valid_0's l1: 0.187575
[3949]	valid_0's l1: 0.187578
[3950]	valid_0's l1: 0.18758
[3951]	valid_0's l1: 0.187583
[3952]	valid_0's l1: 0.187582
[3953]	valid_0's l1: 0.187584
[3954]	valid_0's l1: 0.187585
[3955]	valid_0's l1: 0.187584
[3956]	valid_0's l1: 0.187578
[3957]	valid_0's l1: 0.187579
[3958]	valid_0's l1: 0.187578
[3959]	valid_0's l1: 0.187575
[3960]	valid_0's l1: 0.187575
[3961]	valid_0's l1: 0.187574
[3962]	valid_0's l1: 0.187571
[3963]	valid_0's l1: 0.187573
[3964]	valid_0's l1: 0.187572
[3965]	valid_0's l1: 0.18757
[3966]	valid_0's l1: 0.187568
[3967]	valid_0's l1: 0.187566
[3968]	valid_0's l1: 0.187561
[3969]	valid_0's l1: 0.187556
[3970]	valid_0's l1: 0.187555
[3971]	valid_0's l1: 0.187556
[3972]	valid_0's l1: 0.187547
[3973]	valid_0's l1: 0.187545
[3974]	valid_

[4217]	valid_0's l1: 0.187365
[4218]	valid_0's l1: 0.187363
[4219]	valid_0's l1: 0.187363
[4220]	valid_0's l1: 0.187363
[4221]	valid_0's l1: 0.187368
[4222]	valid_0's l1: 0.187365
[4223]	valid_0's l1: 0.187363
[4224]	valid_0's l1: 0.187368
[4225]	valid_0's l1: 0.187369
[4226]	valid_0's l1: 0.187367
[4227]	valid_0's l1: 0.187369
[4228]	valid_0's l1: 0.187367
[4229]	valid_0's l1: 0.187363
[4230]	valid_0's l1: 0.187362
[4231]	valid_0's l1: 0.18736
[4232]	valid_0's l1: 0.18736
[4233]	valid_0's l1: 0.18736
[4234]	valid_0's l1: 0.18735
[4235]	valid_0's l1: 0.187346
[4236]	valid_0's l1: 0.187346
[4237]	valid_0's l1: 0.187344
[4238]	valid_0's l1: 0.187344
[4239]	valid_0's l1: 0.187343
[4240]	valid_0's l1: 0.187341
[4241]	valid_0's l1: 0.187339
[4242]	valid_0's l1: 0.187336
[4243]	valid_0's l1: 0.187336
[4244]	valid_0's l1: 0.187335
[4245]	valid_0's l1: 0.187335
[4246]	valid_0's l1: 0.187334
[4247]	valid_0's l1: 0.187335
[4248]	valid_0's l1: 0.187334
[4249]	valid_0's l1: 0.187332
[4250]	valid_0

[4492]	valid_0's l1: 0.187142
[4493]	valid_0's l1: 0.187143
[4494]	valid_0's l1: 0.187147
[4495]	valid_0's l1: 0.187145
[4496]	valid_0's l1: 0.187144
[4497]	valid_0's l1: 0.187143
[4498]	valid_0's l1: 0.187142
[4499]	valid_0's l1: 0.187141
[4500]	valid_0's l1: 0.187141
[4501]	valid_0's l1: 0.187144
[4502]	valid_0's l1: 0.187141
[4503]	valid_0's l1: 0.187141
[4504]	valid_0's l1: 0.187136
[4505]	valid_0's l1: 0.187138
[4506]	valid_0's l1: 0.187141
[4507]	valid_0's l1: 0.187145
[4508]	valid_0's l1: 0.187139
[4509]	valid_0's l1: 0.18714
[4510]	valid_0's l1: 0.187137
[4511]	valid_0's l1: 0.187137
[4512]	valid_0's l1: 0.187137
[4513]	valid_0's l1: 0.187134
[4514]	valid_0's l1: 0.187132
[4515]	valid_0's l1: 0.187133
[4516]	valid_0's l1: 0.18713
[4517]	valid_0's l1: 0.18713
[4518]	valid_0's l1: 0.187127
[4519]	valid_0's l1: 0.187125
[4520]	valid_0's l1: 0.187127
[4521]	valid_0's l1: 0.18712
[4522]	valid_0's l1: 0.187114
[4523]	valid_0's l1: 0.187111
[4524]	valid_0's l1: 0.187111
[4525]	valid_0

In [16]:
# Sin la mejora:

Y_pred = reg.predict(X_val.values)

f = np.vectorize(math.exp)
Y_pred = f(Y_pred)
Y_val = f(Y_val.values)
mean_absolute_error(Y_val,Y_pred)

470528.4632887971

### Resultados de la primer alternativa:
Podemos ver entonces que cuando llenamos con nulos, el MAE pasa de **466k** a **462k**.